In [ ]:
import os

import fsspec
import s3fs
import xarray as xr

In [ ]:
target_chunks = {'lat': 600, 'lon': 1440, 'time': 1}
temporal_resolution = "daily"
model = "ACCESS-CM2"
variable = "tasmax"
anon = True

In [ ]:
s3_path = f"s3://nex-gddp-cmip6/NEX-GDDP-CMIP6/{model}/historical/r1i1p1*/{variable}/*"
fs = fsspec.filesystem("s3", anon=anon, skip_instance_cache=False)

In [ ]:
file_paths = fs.glob(s3_path)
fileset = [fs.open(file) for file in file_paths[0:2]]
fileset

In [ ]:
data = xr.open_mfdataset(fileset, combine='by_coords', chunks=target_chunks)
data

In [ ]:
if target_chunks['time'] == 29:
    store_name = f"{variable}_day_{model}_historical_r1i1p1f1_gn/data-0a"
elif target_chunks['time'] == 1:
    store_name = f"{variable}_day_{model}_historical_r1i1p1f1_gn/data-1a"
bucket_path = 's3://carbonplan-benchmarks/data'
dataset_path = '/'.join(file_paths[0].split('/')[1:-1])
target_root = f"NEX-GDDP-CMIP6/ACCESS-CM2/historical/r1i1p1f1/tasmax/tasmax_day_ACCESS-CM2_historical_r1i1p1f1_gn"
target_store = os.path.join(bucket_path, dataset_path, store_name)
target_store

In [ ]:
data.to_zarr(target_store, mode='w')